In [1]:
import signac
import numpy as np
import matplotlib as plt
import freud
import gsd
import hoomd
import gsd.pygsd
import gsd.hoomd
import scipy
from scipy import signal
import time
import os

In [2]:
ptb7_proj = signac.get_project("ptb7-project")
itic_proj = signac.get_project("ptb7-project")
ptb7_proj

Project.get_project('/home/emilyepstein/projects/ptb7-itic/ptb7-project')

In [3]:
print(ptb7_proj.detect_schema())

print(ptb7_proj.detect_schema)['density']

{
 'beats_cut': 'int([-9881, -9681, -9661, ..., -4000, -3800], 35)',
 'density': 'float([0.6, 0.7, 0.8, ..., 1.15, 1.2], 11)',
 'dt': 'float([0.001], 1)',
 'e_factor': 'float([0.5], 1)',
 'kT_reduced': 'float([0.8, 0.9, 1.0, ..., 2.3, 2.4], 17)',
 'molecule': 'str([PTB7_10mer_smiles, PTB7_15mer_smiles, PTB7_5mer_smiles], 3)',
 'n_compounds': 'int([66, 100, 132, ..., 1000, 2000], 9)',
 'n_steps': 'float([10000000.0, 20000000.0, 23000000.0, ..., 100000000.0, 500000000.0], 27)',
 'original_id': 'str([001d1416fedf98dea731e30b7d928762, 002969c1df9d89b2a01e0d396a12b8ea, 00f48f6b7d2d1c502a270b3025a51e5c, ..., ff9e18a2eab6f6742d7a3fc01374cf4f, ffb19d6519b8334b0747dad0568ed9ea], 560)',
 'process': 'str([anneal, quench], 2)',
 'remove_hydrogens': 'bool([True], 1)',
 'save': 'str([false, true], 2)',
 'size': 'str([large, medium, small], 3)',
 'tau': 'int([1], 1)',
}
<bound method Project.detect_schema of Project.get_project('/home/emilyepstein/projects/ptb7-itic/ptb7-project')>


TypeError: 'NoneType' object is not subscriptable

In [4]:
state_dict = {"process": "quench", "size": "medium"}

#= {"molecule": "PTB7_10mer_smiles",  
#   "n_compounds": 200,
#   "density": 0.9,
#   "e_factor": 0.5,
#   "kT_reduced": 1.7,
#   "tau": 1,
#   "n_steps": 60000000.0,
#   "dt": 0.001,
#   "original_id": "ecca9fd02b6f816abf804563103b7ef8",
#   "process": "quench",
#   "size": "medium"}

job_list = ptb7_proj.find_jobs(state_dict)

In [5]:
job_list



JobsCursor(project=Project.get_project('/home/emilyepstein/projects/ptb7-itic/ptb7-project'), filter={'process': 'quench', 'size': 'medium'}, doc_filter=None)

In [7]:
for job in job_list:
    path = (job.fn("trajectory.gsd"))
    print(path)
    rdf_from_gsd(job)
    

/home/emilyepstein/projects/ptb7-itic/ptb7-project/workspace/2ca11a0629e2a4180e303f29c9880bb6/trajectory.gsd


PermissionError: [Errno 13] Permission denied: '/home/emilyepstein/projects/ptb7-itic/ptb7-project/workspace/2ca11a0629e2a4180e303f29c9880bb6/trajectory.gsd'

In [ ]:
def data_analysis(jobs, function = get_rdf_data()):
    for job in jobs:
        job_path = job.workspace()
        if not job.isfile('rdf-data.txt'):
            rdf_results = rdf_from_gsd(job) 
            txt_file = "{}/rdf-data.txt".format(job.workspace())
            x = rdf.bin_centers
            y = rdf.rdf
            np.savetxt(gsd_file, np.transpose([x,y]), delimiter=',', header= "bin_centers, rdf")
#the last 4 lines were originally in the next cell    

In [6]:
def atom_type_pos(frame, atom_types): 
    positions = []
    for idx, type_id in enumerate(frame.particles.typeid):
        if frame.particles.types[type_id] in atom_types:
            positions.append(frame.particles.position[idx])
    return positions


def rdf_from_gsd(job, atom_types='ss', r_max=5, r_min=0.5, nbins=50, start=100):
    gsd_file = "{}/trajectory.gsd".format(job.workspace())
    f = gsd.pygsd.GSDFile(open(gsd_file, "rb"))
    trajectory = gsd.hoomd.HOOMDTrajectory(f)
    
    if r_max is None:
        r_max = max(trajectory[-1].configuration.box[:3]) * 0.45
    freud_rdf = freud.density.RDF(bins=nbins, r_max=r_max, r_min=r_min)
    for frame in trajectory[start:]:
        if atom_types == 'all':
            freud_rdf.compute(frame, reset=False)
        else:
            query_points = atom_type_pos(frame, atom_types)
            box = frame.configuration.box
            freud_rdf.compute(system=(box, query_points), reset=False)
 
        txt_file = "{}/rdf-data.txt".format(job.workspace())
        x = freud_rdf.bin_centers
        y = freud_rdf.rdf
        np.savetxt(gsd_file, np.transpose([x,y]), delimiter=',', header= "bin_centers, rdf")


    f.close()
    return freud_rdf




#we want to do
#def rdf(job):
#    calc rdf
#    save to a text file
#    save file in jobs workspace
    
    
#def msd(job):
#    slope using last x amount of steps
#    save vale of slope to job state_point_dict


In [ ]:
def get_rdf_data(gsd_file, rdf_type):
    data = np.genfromtxt(file, names=True, delimiter=',')
    x = data['{}_x'.format(rdf_type)]
    y = data['{}_y'.format(rdf_type)]
    return x,y